**In case of problems or questions, please first check the list of [Frequently Asked Questions (FAQ)](https://stardist.net/docs/faq.html).**

Please shutdown all other training/prediction notebooks before running this notebook (as those might occupy the GPU memory otherwise).

Environment creation:
```bash
mamba create -n stardist-napari-bioio-env -c conda-forge python=3.10 napari pyqt
mamba activate stardist-napari-bioio-env
mamba install -c conda-forge cudatoolkit=11.2 cudnn=8.1.0
python -m pip install "tensorflow<2.11"
pip install numpy==1.26.4 # avoid having numpy <= 2.0
python -c "import tensorflow as tf; print(tf.config.list_physical_devices('GPU'))" # check if GPU is available
pip install stardist
pip install bioio bioio-czi
pip install stardist-napari
```

# NOTE

**If you have not looked at the [regular example notebooks](../2D), please do so first.**  
The notebooks in this folder provide further details about the inner workings of StarDist and might be useful if you want to apply it in a slightly different context.

In [1]:
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
%config InlineBackend.figure_format = 'retina'

from tifffile import imread, imwrite as imsave
from csbdeep.utils import Path, normalize
from csbdeep.utils.tf import keras_import
keras = keras_import()

from stardist import export_imagej_rois, random_label_cmap
from stardist.models import StarDist2D

np.random.seed(0)
cmap = random_label_cmap()

In [ ]:
import tensorflow as tf
physical_devices = tf.config.list_physical_devices('GPU')
print(physical_devices)

# Data

In [3]:
def show_image(img, **kwargs):
    """Plot large image at different resolutions."""
    fig, ax = plt.subplots(2,4, figsize=(16,8))
    mid = [s//2 + 600 for s in img.shape[:2]]
    for a,t,u in zip(ax.ravel(),[1,2,4,8,16,32,64,128],[16,8,4,2,1,1,1,1]):
        sl = tuple(slice(c - s//t//2, c + s//t//2, u) for s,c in zip(img.shape[:2],mid))
        a.imshow(img[sl], **kwargs)
        a.axis('off')
    plt.tight_layout()
    plt.show()

Download example image in [SVS format](http://paulbourke.net/dataformats/svs/), which in this case can be read as a TIFF file.

In [4]:
datadir  = Path('data') / 'big'
# filename = 'image.tif'
# url      = 'https://pathology.cancerimagingarchive.net/download/SAR/C3L-00035-21.svs'

# datadir.mkdir(parents=True, exist_ok=True)
# if not (datadir/filename).exists():
#     keras.utils.get_file(filename, url, cache_dir=str(datadir), cache_subdir='.')
# None;

In [5]:
from bioio import BioImage
import bioio_czi
import numpy as np
from skimage import io

path = r"D:\Datasets\Julieta\ImageAnalysis-BFX_VisiumHD\20240604-HD-LAB5407-CaminDean\2024_06_04__BF_0001.czi"
path = r"D:\Datasets\Julieta\ImageAnalysis-BFX_VisiumHD\20240403-HD-LAB5278-YungHaeK\2024_04_03_0001.czi"

img = BioImage(path, reader=bioio_czi.Reader)
img = np.squeeze(img.data)

image = np.squeeze(img.data)
background_mask = np.all(np.squeeze(img.data) == [0, 0, 0], axis=-1)
image[background_mask] = [158, 158, 158] # add light gray background to black areas


# path = r"C:\Users\mazo260d\Documents\GitHub\stardist\2024_06_04__BF_0001_with_artificial_light_BG.tiff"
# img = io.imread(path)

In [ ]:
# img = imread(str(datadir/filename))
print(f"image of shape {image.shape} and dtype {image.dtype}")

In [ ]:
show_image(image)

# Prediction

**NOTE:** Although this example uses 2D images, the demonstrated functionality also works for 3D StarDist.

## Model

Load pretrained model for [H&E stained](https://en.wikipedia.org/wiki/H%26E_stain) images.

In [ ]:
model = StarDist2D.from_pretrained('2D_versatile_he')

## Image normalization

StarDist expects input images to be normalized or that a suitable `normalizer` is passed to `model.predict_instances`.  
Typically, you'd normalize an input image directly without the need for an additional `normalizer` (commented out):

In [9]:
# img = normalize(img, 1, 99.8)
# normalizer = None

However, this can be slow for large images, or even infeasible for images that do not fit in memory (e.g. via [Zarr](https://zarr.readthedocs.io), see below).  
Hence, we do not normalize the input image `img` and instead define a custom `normalizer`:

In [10]:
from csbdeep.data import Normalizer, normalize_mi_ma

class MyNormalizer(Normalizer):
    def __init__(self, mi, ma):
            self.mi, self.ma = mi, ma
    def before(self, x, axes):
        return normalize_mi_ma(x, self.mi, self.ma, dtype=np.float32)
    def after(*args, **kwargs):
        assert False
    @property
    def do_after(self):
        return False

# mi, ma = np.percentile(img[::8], [1,99.8])                      # compute percentiles from low-resolution image
# mi, ma = np.percentile(img[13000:16000,13000:16000], [1,99.8])  # compute percentiles from smaller crop
mi, ma = 0, 255                                                   # use min and max dtype values (suitable here)
normalizer = MyNormalizer(mi, ma)

## Block-wise prediction

We could now run the prediction like this (commented out):

In [11]:
# labels, polys = model.predict_instances(img, normalizer=normalizer, n_tiles=(32,32,1))

This would break down the input image into 1024 overlapping _tiles_ to be processed individually by the CNN, but the subsequent non-maximum suppression step would be run on the entire image all at once. This can be prohibitive both in terms of memory and computation requirements.

Hence, we can use `model.predict_instances_big` instead, which will break up the input image into larger overlapping _blocks_, each of which is processed via `model.predict_instances`. Please see the documentation:

In [ ]:
help(model.predict_instances_big)

Finally, we call `model.predict_instances_big` to run the block-wise prediction:

In [ ]:
labels, polys = model.predict_instances_big(image, axes='YXC', block_size=4096, min_overlap=128, context=128,
                                            normalizer=normalizer, n_tiles=(4,4,1))

## Show & Save

Showing the predicted label image in the same way as the input image above:

In [ ]:
show_image(labels, cmap=cmap)

**Bonus:** Since label ids are consecutive within each block, they are quantized to the same color when using a colormap with few colors. This allows us to visualize the blocks that have been used during prediction (left). Prediction blocks are not visible when using a suitable colormap for label images (right):

In [ ]:
fig, (a,b) = plt.subplots(1,2, figsize=(16,16))
a.imshow(labels[::8,::8], cmap='tab20b')
b.imshow(labels[::8,::8], cmap=cmap)
a.axis('off'); b.axis('off');
None;

In [19]:
import napari
from napari.utils import nbscreenshot

In [17]:
viewer = napari.Viewer()

In [ ]:
viewer.add_image(image)
viewer.add_labels(labels)
nbscreenshot(viewer)

We can save the results to disk as label image and/or ImageJ ROIs. (Compression highly recommended.)

In [24]:
from zipfile import ZIP_DEFLATED

imsave(str(datadir/'labels2.tif'), labels, compression=ZIP_DEFLATED)
# export_imagej_rois(str(datadir/'labels_roi.zip'), polys['coord'], compression=ZIP_DEFLATED)

# Zarr instead of Numpy arrays for very large images

If your images are too large to be loaded all at once, you can use [Zarr](https://zarr.readthedocs.io) to store and process them.

In [15]:
import zarr

# for this demo: make a zarr array copy of img
zarr.save_array(str(datadir/'image.zarr'), img)

Load input image as read-only Zarr array:

In [ ]:
img = zarr.open(str(datadir/'image.zarr'), mode='r')
img.info

Define the output label image as an empty Zarr array:

In [ ]:
labels = zarr.open(str(datadir/'labels.zarr'), mode='w', shape=img.shape[:2], chunks=img.chunks[:2], dtype=np.int32)
labels.info

Run the block-wise prediction and pass `labels_out=labels` to write to the predefined Zarr array for the label image.  
(Note: You can alternatively pass `labels_out=False` if you don't need the label image.)

In [ ]:
labels, polys = model.predict_instances_big(img, axes='YXC', block_size=4096, min_overlap=128, context=128,
                                            normalizer=normalizer, n_tiles=(4,4,1),
                                            labels_out=labels)

The label image is now populated:

In [ ]:
labels.info